In [3]:
cfg = {
    'dev_train_len': 25*10**3,
    'dev_validation_len': 5*10**3,
    'learning_rate': 0.001,
    'epochs': 100,
    'embedding_dim': 50,
    'batch_size': 32,
    'dropout': 0,
    'optimizer': 'Adam',
    'num_layers': 2
}

cfg['hidden_dim'] = cfg['embedding_dim']

In [4]:
from nltk import word_tokenize
from gensim.models import KeyedVectors
import gensim.downloader as api
import unicodedata
import random
import torch
import re

def normalize_unicode(text: str) -> str:
    return unicodedata.normalize('NFD', text)

def tokenize_corpus(s: str) -> str:
    s = normalize_unicode(s)
    s = s.lower()
    s = re.sub(r"""[^a-zA-Z0-9?.,;'"]+""", " ", s)
    s = re.sub(r'(.)\1{3,}',r'\1', s)
    s = s.rstrip().strip()
    return s

def get_word_tokenized_corpus(s: str) -> list:
    return word_tokenize(s)

glove_dict = {
    '50': 'glove-wiki-gigaword-50',
    '100': 'glove-wiki-gigaword-100',
    '200': 'glove-wiki-gigaword-200'
}

glove_dict['50'] = api.load(glove_dict['50'])
# glove_dict['100'] = api.load(glove_dict['100'])
# glove_dict['200'] = api.load(glove_dict['200'])

def create_vocab(sentences: list, embedding_dim: int):
    glove = glove_dict[str(embedding_dim)]

    Emb = KeyedVectors(vector_size=glove.vector_size)
    vocab = []

    for sentence in sentences:
        vocab.extend(sentence)

    vocab = set(vocab)

    vectors, keys = [], []
    for token in vocab:
        if token in glove:
            vectors.append(torch.tensor(glove[token]))
            keys.append(token)

    keys.extend(['<unk>', '<pad>', '<sos>', '<eos>'])
    vectors.append(torch.mean(torch.stack(vectors), dim=0).numpy())
    vectors.append([0 for _ in range(embedding_dim)])
    vectors.append([random.random() for _ in range(embedding_dim)])
    vectors.append([random.random() for _ in range(embedding_dim)])
    Emb.add_vectors(keys, vectors)

    return Emb

def get_sentence_index(sentence: list, Emb: KeyedVectors):
    word_vec = []

    word_vec.append(Emb.key_to_index['<sos>'])
    for word in sentence:
        word_vec.append(get_vocab_index(word, Emb))
    word_vec.append(Emb.key_to_index['<eos>'])

    return torch.tensor(word_vec)

def get_vocab_index(word: str, Emb: KeyedVectors):
    if word in Emb:
        return Emb.key_to_index[word]
    return Emb.key_to_index['<unk>']


In [5]:
import numpy as np
import torch
import pandas as pd
import os

In [6]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8" # ":4096:2"
else:
    DEVICE = torch.device('cpu')
print(DEVICE)

cuda


In [7]:
DEV_TRAIN_LEN = cfg['dev_train_len']
DEV_VALIDATION_LEN = cfg['dev_validation_len']

DIR = '/scratch/shu7bh/RES/PRE'

In [8]:
import os
if not os.path.exists(DIR):
    os.makedirs(DIR)

In [9]:
print(DEV_TRAIN_LEN)
print(DEV_VALIDATION_LEN)

25000
5000


In [10]:
df = pd.read_csv('data/train.csv')
df = df.sample(frac=1, random_state=0).reset_index(drop=True)
df['Description'] = df['Description'].apply(tokenize_corpus)
df['Description'] = df['Description'].apply(get_word_tokenized_corpus)

In [11]:
df['Class Index']

0         4
1         4
2         4
3         1
4         3
         ..
119995    4
119996    3
119997    1
119998    4
119999    4
Name: Class Index, Length: 120000, dtype: int64

In [12]:
dev_train = df[:DEV_TRAIN_LEN]['Description']
dev_validation = df[DEV_TRAIN_LEN:DEV_TRAIN_LEN + DEV_VALIDATION_LEN]['Description']

In [13]:
dev_validation

25000    [linux, is, gaining, ground, with, companies, ...
25001    [google, ,, the, internet, search, engine, ,, ...
25002    [ap, defending, champion, tiger, woods, said, ...
25003    [advanced, micro, devices, will, add, quot, ;,...
25004    [reuters, about, 100, tibetan, exiles, chanted...
                               ...                        
29995    [they, were, 4, 12, last, year, ., they, have,...
29996    [luke, donald, and, paul, casey, are, unlikely...
29997    [yahoo, is, eager, to, improve, its, web, base...
29998    [ankara, strasbourg, ,, france, reuters, prime...
29999    [leipzig, game, convention, in, germany, ,, th...
Name: Description, Length: 5000, dtype: object

In [14]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [15]:
import torch
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class ELMO(nn.Module):
    def __init__(self, Emb, hidden_dim, dropout, num_layers):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(Emb.vectors), padding_idx=Emb.key_to_index['<pad>'])
        self.lstm = nn.LSTM(Emb.vectors.shape[1], hidden_dim, batch_first=True, bidirectional=True, num_layers=num_layers, dropout=dropout)

    def forward(self, X, X_lengths):
        X = self.embedding(X)
        X = pack_padded_sequence(X, X_lengths, batch_first=True, enforce_sorted=False)
        X, (h_n, c_n) = self.lstm(X, None)
        X, _ = pad_packed_sequence(X, batch_first=True)
        return X, h_n, c_n

In [16]:
from torch import nn

class LM(nn.Module):
    def __init__(self, Emb, hidden_dim, dropout, num_layers):
        super().__init__()

        self.elmo = ELMO(Emb, hidden_dim, dropout, num_layers)
        self.fc = nn.Linear(hidden_dim * 2, Emb.vectors.shape[0])

    def forward(self, X, X_lengths):
        X, _, _ = self.elmo(X, X_lengths)
        X = self.fc(X)
        return X

In [17]:
from torch.utils.data import Dataset
import torch

class SentencesDataset(Dataset):
    def __init__(self, sentences: list, Emb):
        super().__init__()

        self.data = []
        for sentence in sentences:
            self.data.append(get_sentence_index(sentence, Emb))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], torch.tensor(len(self.data[idx]))

In [18]:
class Collator:
    def __init__(self, Emb):
        self.pad_index = Emb.key_to_index['<pad>']

    def __call__(self, batch):
        X, X_lengths = zip(*batch)
        X = pad_sequence(X, batch_first=True, padding_value=self.pad_index)
        return X[:, :-1], X[:, 1:], torch.stack(X_lengths) - 1

In [19]:
import tqdm

def fit(model, dataloader, train, es, loss_fn, optimizer):
    model.train() if train else model.eval()
    epoch_loss = []

    pbar = tqdm.tqdm(dataloader)

    for X, Y, X_lengths in pbar:
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)

        Y_pred = model(X, X_lengths)
        Y_pred = Y_pred.reshape(-1, Y_pred.shape[2])

        Y = Y.reshape(-1)

        loss = loss_fn(Y_pred, Y)
        epoch_loss.append(loss.item())

        X.detach()
        Y_pred.detach()
        Y.detach()

        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        pbar.set_description(f'{"T" if train else "V"} Loss: {loss.item():7.4f}, Avg Loss: {np.mean(epoch_loss):7.4f}, Best Loss: {es.best_loss:7.4f}, Counter: {es.counter}')

    return np.mean(epoch_loss)

In [20]:
import numpy as np

class EarlyStopping:
    def __init__(self, patience:int = 3, delta:float = 0.001):
        self.patience = patience
        self.counter = 0
        self.best_loss:float = np.inf
        self.best_model_pth = 0
        self.delta = delta

    def __call__(self, loss, epoch: int):
        should_stop = False

        if loss >= self.best_loss - self.delta:
            self.counter += 1
            if self.counter > self.patience:
                should_stop = True
        else:
            self.best_loss = loss
            self.counter = 0
            self.best_model_pth = epoch
        return should_stop

In [21]:
def train(EPOCHS, model, training_dataloader, validation_dataloader, loss_fn, optimizer):
    es = EarlyStopping(patience=3, delta=0.001)

    for epoch in range(EPOCHS):
        print(f'\nEpoch {epoch+1}')

        epoch_loss = fit(model, training_dataloader, True, es, loss_fn, optimizer)

        with torch.no_grad():
            epoch_loss = fit(model, validation_dataloader, False, es, loss_fn, optimizer)
            if es(epoch_loss, epoch):
                break
            if es.counter == 0:
                torch.save(model.state_dict(), os.path.join(DIR, f'best_model.pth'))
                torch.save(model.elmo.state_dict(), os.path.join(DIR, f'best_model_elmo.pth'))

In [22]:
def run(config=None):
    BATCH_SIZE = cfg['batch_size']

    HIDDEN_DIM = config['hidden_dim']
    DROP_OUT = config['dropout']
    OPTIMIZER = config['optimizer']
    LEARNING_RATE = config['learning_rate']
    EPOCHS = config['epochs']
    EMBEDDITNG_DIM = config['embedding_dim']
    NUM_LAYERS = config['num_layers']

    Emb = create_vocab(df['Description'], EMBEDDITNG_DIM)

    dev_train_dataset = SentencesDataset(dev_train, Emb)
    dev_validation_dataset = SentencesDataset(dev_validation, Emb)

    collate_fn = Collator(Emb)

    training_dataloader = DataLoader(dev_train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=4)
    validation_dataloader = DataLoader(dev_validation_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=4)

    torch.cuda.empty_cache()

    model = LM(Emb, HIDDEN_DIM, DROP_OUT, NUM_LAYERS).to(DEVICE)

    optimizer = getattr(torch.optim, OPTIMIZER)(model.parameters(), lr=LEARNING_RATE)
    loss_fn = nn.CrossEntropyLoss(ignore_index=Emb.key_to_index['<pad>'])

    train(EPOCHS, model, training_dataloader, validation_dataloader, loss_fn, optimizer)

run(cfg)


Epoch 1


T Loss:  5.6669, Avg Loss:  6.7665, Best Loss:     inf, Counter: 0: 100%|██████████| 782/782 [00:25<00:00, 30.71it/s]
V Loss:  5.8964, Avg Loss:  5.5419, Best Loss:     inf, Counter: 0: 100%|██████████| 157/157 [00:02<00:00, 70.91it/s]



Epoch 2


T Loss:  3.2303, Avg Loss:  4.5110, Best Loss:  5.5419, Counter: 0: 100%|██████████| 782/782 [00:24<00:00, 31.70it/s]
V Loss:  3.8009, Avg Loss:  3.7043, Best Loss:  5.5419, Counter: 0: 100%|██████████| 157/157 [00:02<00:00, 68.08it/s]



Epoch 3


...

T Loss:  0.0090, Avg Loss:  0.0085, Best Loss:  0.3846, Counter: 1: 100%|██████████| 782/782 [00:24<00:00, 31.85it/s]
V Loss:  0.5817, Avg Loss:  0.3875, Best Loss:  0.3846, Counter: 1: 100%|██████████| 157/157 [00:02<00:00, 67.62it/s]



Epoch 28


T Loss:  0.0071, Avg Loss:  0.0090, Best Loss:  0.3846, Counter: 2: 100%|██████████| 782/782 [00:24<00:00, 31.91it/s]
V Loss:  0.3678, Avg Loss:  0.3871, Best Loss:  0.3846, Counter: 2: 100%|██████████| 157/157 [00:02<00:00, 68.34it/s]



Epoch 29


T Loss:  0.0069, Avg Loss:  0.0060, Best Loss:  0.3846, Counter: 3: 100%|██████████| 782/782 [00:24<00:00, 31.88it/s]
V Loss:  0.3085, Avg Loss:  0.3859, Best Loss:  0.3846, Counter: 3: 100%|██████████| 157/157 [00:02<00:00, 68.14it/s]


In [22]:
import tqdm

def run_epoch(model, dataloader, loss_fn):
    epoch_loss = []

    pbar = tqdm.tqdm(dataloader)

    for X, Y, X_lengths in pbar:
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)

        Y_pred = model(X, X_lengths)
        Y_pred = Y_pred.reshape(-1, Y_pred.shape[2])

        Y = Y.reshape(-1)

        loss = loss_fn(Y_pred, Y)
        epoch_loss.append(loss.item())

        pbar.set_description(f'Loss: {loss.item():7.4f}, Avg Loss: {np.mean(epoch_loss):7.4f}')

    return np.mean(epoch_loss)

In [23]:
def validate(elmo, validation_dataloader, loss_fn):
    with torch.no_grad():
        elmo.eval()
        epoch_loss = run_epoch(elmo, validation_dataloader, loss_fn)
        print(f'Validation Loss: {epoch_loss:7.4f}')

In [24]:
Emb = create_vocab(df['Description'], cfg['embedding_dim'])

dev_validation_dataset = SentencesDataset(dev_validation, Emb)

collate_fn = Collator(Emb)

validation_dataloader = DataLoader(dev_validation_dataset, batch_size=cfg['batch_size'], shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=4)
model = LM(Emb, cfg['hidden_dim'], cfg['dropout'], cfg['num_layers']).to(DEVICE)

model.load_state_dict(torch.load(os.path.join(DIR, 'best_model.pth')))

validate(model, validation_dataloader, nn.CrossEntropyLoss(ignore_index=Emb.key_to_index['<pad>']))

Loss:  0.5460, Avg Loss:  0.3803: 100%|██████████| 313/313 [00:02<00:00, 108.19it/s]

Validation Loss:  0.3803


In [48]:
elmo = ELMO(Emb, cfg['hidden_dim'], cfg['dropout'], cfg['num_layers']).to(DEVICE)

# load only elmo from the best model
elmo.load_state_dict(torch.load(os.path.join(DIR, 'best_model_elmo.pth')))

<All keys matched successfully>

In [49]:
from torch.utils.data import DataLoader, Dataset

class DownStreamDataset(Dataset):
    def __init__(self, df: pd.DataFrame, Emb: KeyedVectors):
        self.descriptions = df['Description']
        self.descriptions = [get_sentence_index(description, Emb) for description in self.descriptions]
        # self.df.loc[: 'Description'] = df['Description'].apply(get_sentence_index, Emb=Emb)
        self.class_index = list(c - 1 for c in df['Class Index'])
        self.Emb = Emb

    def __len__(self):
        return len(self.class_index)

    def __getitem__(self, idx):
        l = torch.tensor(len(self.descriptions[idx]))
        return self.descriptions[idx], l, torch.tensor(self.class_index[idx])

In [50]:
class DownStreamCollator:
    def __init__(self, Emb):
        self.pad_index = Emb.key_to_index['<pad>']

    def __call__(self, batch):
        X, X_lengths, Y = zip(*batch)
        X = pad_sequence(X, batch_first=True, padding_value=self.pad_index)
        return X, torch.stack(X_lengths), torch.stack(Y)

In [51]:
downstream_train = df[DEV_TRAIN_LEN + DEV_VALIDATION_LEN:3 * DEV_TRAIN_LEN]
downstream_validation = df[:3 * DEV_VALIDATION_LEN]

downstream_train_dataset = DownStreamDataset(downstream_train, Emb)
downstream_validation_dataset = DownStreamDataset(downstream_validation, Emb)

downstream_collate_fn = DownStreamCollator(Emb)

downstream_training_dataloader = DataLoader(downstream_train_dataset, batch_size=64, shuffle=True, collate_fn=downstream_collate_fn, pin_memory=True, num_workers=4)
downstream_validation_dataloader = DataLoader(downstream_validation_dataset, batch_size=64, shuffle=True, collate_fn=downstream_collate_fn, pin_memory=True, num_workers=4)

In [73]:
class DownStream(nn.Module):
    def __init__(self, elmo, dropout, delta = None):
        super().__init__()
        self.elmo = elmo
        # freeze the ELMO parameters
        for param in self.elmo.parameters():
            param.requires_grad = False

        self.hidden_dim = self.elmo.hidden_dim
        self.num_layers = self.elmo.num_layers
        self.dropout = dropout

        if delta is None:
            self.delta = nn.Parameter(torch.randn(1, self.num_layers + 1))
        else:
            self.delta = delta.to(DEVICE)

        self.linear = nn.Linear(self.hidden_dim * 2, 4)

    def forward(self, X, X_lengths):
        _, Y1, Y2 = self.elmo(X, X_lengths)

        Y = torch.mean(torch.stack([Y1, Y2]), dim=0)
        Y = Y.permute(1, 0, 2).reshape(Y.shape[1], self.num_layers, self.hidden_dim * 2)

        X = self.elmo.embedding(X)
        X = torch.mean(X, dim=1, dtype=torch.float32)
        X = torch.cat([X, X], dim=1)
        X = X.unsqueeze(1)

        Y = torch.cat([X, Y], dim=1)

        Y = (self.delta / torch.sum(self.delta) ) @ Y

        Y = torch.sum(Y, dim=1)

        Y = self.linear(Y)

        return Y

In [87]:
def downstream_fit(model, dataloader, train, loss_fn, optimizer, es):
    model.train() if train else model.eval()
    epoch_loss = []

    pbar = tqdm.tqdm(dataloader)

    for X, X_lengths, Y in pbar:
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)

        Y_pred = model(X, X_lengths)
        Y_pred = Y_pred.reshape(-1, Y_pred.shape[-1])

        loss = loss_fn(Y_pred, Y)
        epoch_loss.append(loss.item())

        X.detach()
        Y_pred.detach()
        Y.detach()

        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        pbar.set_description(f'{"T" if train else "V"} Loss: {loss.item():7.4f}, Avg Loss: {np.mean(epoch_loss):7.4f}, Best Loss: {es.best_loss:7.4f}, Counter: {es.counter}')

    return np.mean(epoch_loss)

In [88]:
dloss_fn = nn.CrossEntropyLoss()

def downstream_train_fn(EPOCHS, model, training_dataloader, validation_dataloader, loss_fn, optimizer):
    es = EarlyStopping(patience=3, delta=0.001)
    for epoch in range(EPOCHS):
        print(f'\nEpoch {epoch+1}')

        epoch_loss = downstream_fit(model, training_dataloader, True, loss_fn, optimizer, es)
        print(f'Train Loss: {epoch_loss:7.4f}')
        # wandb.log({'downstream_train_loss': epoch_loss})

        with torch.no_grad():
            epoch_loss = downstream_fit(model, validation_dataloader, False, loss_fn, optimizer, es)
            print(f'Validation Loss: {epoch_loss:7.4f}')
            if es(epoch_loss, epoch):
                break
            if es.counter == 0:
                torch.save(model.state_dict(), os.path.join(DIR, f'downstream_best_model.pth'))
            # wandb.log({'downstream_validation_loss': epoch_loss})

In [76]:
tdf = pd.read_csv('data/test.csv')
tdf = tdf.sample(frac=1, random_state=0).reset_index(drop=True)
tdf['Description'] = tdf['Description'].apply(tokenize_corpus)
tdf['Description'] = tdf['Description'].apply(get_word_tokenized_corpus)

In [89]:
downstream_train_dataset = DownStreamDataset(tdf, Emb)
downstream_test_dataloader = DataLoader(downstream_train_dataset, batch_size=64, shuffle=True, collate_fn=downstream_collate_fn, pin_memory=True, num_workers=4)

In [90]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def downstream_test_metrics(model, dataloader, loss_fn):
    epoch_loss = []

    pbar = tqdm.tqdm(dataloader)

    Y_pred_all = []
    Y_all = []
    Y_len = 0

    for X, X_lengths, Y in pbar:
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)

        Y_pred = model(X, X_lengths)
        Y_pred = Y_pred.reshape(-1, Y_pred.shape[-1])

        Y_all.extend(Y.tolist())
        Y_pred_all.extend(torch.argmax(Y_pred, dim=1).tolist())
        Y_len += len(Y)

        loss = loss_fn(Y_pred, Y)
        epoch_loss.append(loss.item())

        X.detach()
        Y_pred.detach()
        Y.detach()

        pbar.set_description(f'{"T" if train else "V"} Loss: {loss.item():7.4f}, Avg Loss: {np.mean(epoch_loss):7.4f}')

    accuracy = accuracy_score(Y_all, Y_pred_all)
    print(f'Accuracy: {accuracy:7.4f}')
    print(confusion_matrix(Y_all, Y_pred_all))
    print(classification_report(Y_all, Y_pred_all, digits=4))

    return np.mean(epoch_loss)

In [93]:
delta = [None, [3, 0, 0], [1, 2, 3], [0, 0, 3], [1, 1, 1]] 
for i in range(1, len(delta)):
    delta[i] = torch.tensor(delta[i], dtype=torch.float32).reshape(1, -1).to(DEVICE)
    print(delta[i])
    print(delta[i].shape)

for d in delta:
    dmodel = DownStream(elmo, cfg['dropout'], d).to(DEVICE)
    doptimizer = getattr(torch.optim, cfg['optimizer'])(dmodel.parameters(), lr=cfg['learning_rate'])
    print(f'\n\nDelta: {d}')
    downstream_train_fn(cfg['epochs'], dmodel, downstream_training_dataloader, downstream_validation_dataloader, dloss_fn, doptimizer)
    dmodel.load_state_dict(torch.load(os.path.join(DIR, 'downstream_best_model.pth')))
    with torch.no_grad():
        loss = downstream_test_metrics(dmodel, downstream_test_dataloader, dloss_fn)
        # calculate accuracy and f1 score
        
        print(f'Test loss: {loss}')

tensor([[3., 0., 0.]], device='cuda:0')
torch.Size([1, 3])
tensor([[1., 2., 3.]], device='cuda:0')
torch.Size([1, 3])
tensor([[0., 0., 3.]], device='cuda:0')
torch.Size([1, 3])
tensor([[1., 1., 1.]], device='cuda:0')
torch.Size([1, 3])


Delta: None

Epoch 1


  0%|          | 0/704 [00:00<?, ?it/s]

T Loss:  0.8251, Avg Loss:  1.1084, Best Loss:     inf, Counter: 0: 100%|██████████| 704/704 [00:06<00:00, 104.15it/s]


Train Loss:  1.1084


V Loss:  0.8209, Avg Loss:  0.7772, Best Loss:     inf, Counter: 0: 100%|██████████| 235/235 [00:02<00:00, 111.51it/s]


...

Train Loss:  0.5471


V Loss:  0.4170, Avg Loss:  0.5544, Best Loss:  0.5527, Counter: 3: 100%|██████████| 235/235 [00:02<00:00, 109.27it/s]


Validation Loss:  0.5544


T Loss:  0.5831, Avg Loss:  0.5560: 100%|██████████| 119/119 [00:01<00:00, 94.85it/s] 


Accuracy:  0.7953
[[1574  126  125   75]
 [ 103 1705   42   50]
 [ 131   55 1459  255]
 [ 180   90  324 1306]]
              precision    recall  f1-score   support

           0     0.7918    0.8284    0.8097      1900
           1     0.8629    0.8974    0.8798      1900
           2     0.7482    0.7679    0.7579      1900
           3     0.7746    0.6874    0.7284      1900

    accuracy                         0.7953      7600
   macro avg     0.7944    0.7953    0.7939      7600
weighted avg     0.7944    0.7953    0.7939      7600

Test loss: 0.5559738839373869


In [2]:
import torch
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class CharCNNELMO(nn.Module):
    def __init__(self, Emb, hidden_dim, dropout, num_layers):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.char_embedding = nn.Embedding.from_pretrained(torch.FloatTensor(Emb.vectors), padding_idx=Emb.key_to_index['<pad>'], freeze=False)

        self.conv1 = nn.Conv1d(Emb.vectors.shape[1], hidden_dim, kernel_size=3)

        self.lstm = nn.LSTM(Emb.vectors.shape[1], hidden_dim, batch_first=True, bidirectional=True, num_layers=num_layers, dropout=dropout)

    def forward(self, X, X_lengths):
        X = self.char_embedding(X)
        X = pack_padded_sequence(X, X_lengths, batch_first=True, enforce_sorted=False)
        X, (h_n, c_n) = self.lstm(X, None)
        X, _ = pad_packed_sequence(X, batch_first=True)
        return X, h_n, c_n

In [ ]:
# creat co